[Control] Analysis of the results
--------

This notebook focuses on the analysis of the properties for the control setting.
In particular, when generating sentences from diferent models with different decoding algorithms:

- How do different properties of the model change?
- Do we obtain multimodal distributions?
- Does the 10-th or 90-th percentile analysis tells a different story than the average?

In [ ]:
from typing import List

import pandas as pd
import numpy as np

import os

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
sns.set_palette(sns.color_palette("colorblind"))

## Load Data

This version of the code is limited to processing a single sample (i.e., sampled with a single seed for the same decoding configuration). Bootstrap-like analysis are not being considered in this notebook

In [ ]:
# -----------------------------------------------------------------
# Global constants setup: User should update these variables
# -----------------------------------------------------------------

MODEL = "EleutherAI__pythia-1.4b" 

DECODING_SETUP = [
    "multinomial",
    ("temperature", (0.1, 0.3, 0.5, 1.15, 1.5)),
    ("top_k", (2, 10, 40, 100)),
    ("top_p", (0.1, 0.3, 0.5, 0.7, 0.8, 0.9)),
]

RESULTS_DIR = "/extra/ucinlp1/cbelem/experiment-ro-prompts/generations-results/uncond"

# -----------------------------------------------------------------

def get_filenames(base_dir: str, model: str, decoding_configs: List) -> List[str]:
    """Compute all filenames for a given model and decoding configs."""
    def parse_decoding_algo(dec):
        if isinstance(dec, str):
            return [dec]
        
        elif isinstance(dec, tuple) and len(dec) == 2:
            if dec[0] =="temperature":
                return [f"temperature_{d}" for d in dec[1]]
            if dec[0] =="top_p":
                return [f"top_p_{p}" for p in dec[1]]
            if dec[0] =="top_k":
                return [f"top_k_{k}" for k in dec[1]]
        else:
            raise ValueError(f"Decoding config unknown: {dec}")
            
    decodings = []
    filepaths = []
    
    for configs in decoding_configs:
        for config in parse_decoding_algo(configs):
            fp = f"{base_dir}/{model}/{config}.csv"
            
            if os.path.isfile(fp):
                decodings.append(config)
                filepaths.append(fp)
            else:
                print("FileNotFound:\n-->", fp)
                
    return decodings, filepaths


# --------------------------------------------------
# Read files
# --------------------------------------------------
decodings, filepaths = get_filenames(RESULTS_DIR, MODEL, DECODING_SETUP)
files = [pd.read_csv(fp, index_col=0) for fp in filepaths]

# Discriminate between different decodings
all_files = {"temp": [], "top_p": [], "top_k": []}

for file, decoding in zip(files, decodings):
    file.insert(0, "decoding", decoding)
    
    if decoding == "multinomial":
        all_files["temp"].append(file)
        all_files["top_p"].append(file)
        all_files["top_k"].append(file)
    elif decoding.startswith("temperature"):
        all_files["temp"].append(file)
    elif decoding.startswith("top_k"):
        all_files["top_k"].append(file)
    elif decoding.startswith("top_p"):
        all_files["top_p"].append(file)
    
# all_files: mapping decoding_algo --> dataframe
all_files = {dec: pd.concat(dec_files, axis=0) for dec, dec_files in all_files.items()}
# all_files concatenated
all_files_concat = pd.concat(files, axis=0)

## Properties distribution

In [ ]:
DEFAULT_PALETTE = sns.color_palette("colorblind", n_colors=1)

# fixme - hard coded decoding groupings
TEMP_DECODINGS = ['temperature_0.1', 'temperature_0.3', 'temperature_0.5', 'temperature_1.15', 'temperature_1.5']
TOPK_DECODINGS = ['top_k_2', 'top_k_10', 'top_k_40', 'top_k_100']
TOPP_DECODINGS = ['top_p_0.1', 'top_p_0.3', 'top_p_0.5', 'top_p_0.7', 'top_p_0.8', 'top_p_0.9']

TEMP_PALETTE = sns.color_palette("Greens", n_colors=len(TEMP_DECODINGS))
TOPK_PALETTE = sns.color_palette("Blues", n_colors=len(TOPK_DECODINGS))
TOPP_PALETTE = sns.color_palette("Reds", n_colors=len(TOPP_DECODINGS))

DEFAULT_COLORS = {
    "temp": {c: TEMP_PALETTE[i] for i, c in enumerate(TEMP_DECODINGS)},
    "top_p": {c: TOPP_PALETTE[i] for i, c in enumerate(TOPP_DECODINGS)},
    "top_k": {c: TOPK_PALETTE[i] for i, c in enumerate(TOPK_DECODINGS)},
}
DEFAULT_COLORS["temp"]["multinomial"] = DEFAULT_PALETTE[0]
DEFAULT_COLORS["top_p"]["multinomial"] = DEFAULT_PALETTE[0]
DEFAULT_COLORS["top_k"]["multinomial"] = DEFAULT_PALETTE[0]

In [ ]:
def compute_avgs(data: pd.DataFrame, property1: str, colname: str) -> tuple:
    avgs = data[[colname, property1]].groupby(colname).mean()[property1]
    dec, dec_avg = zip(*avgs.items())
    # returns the name of the decoding algorithm and its
    # decoding_avg
    return dec, dec_avg


def plot_histograms(property1, figsize=(20, 6), **kwargs):
    fig, axes = plt.subplots(1, 3, sharex=True, sharey=True, figsize=figsize)
    fig.suptitle(f"{property1} distribution")
    hist_kwargs = dict(hue="decoding", x=property1, element="poly", alpha=0.3, stat="probability")
    hist_kwargs.update(**kwargs)
    for i, decode_alg in enumerate(("temp", "top_p", "top_k")):
        data, palette = all_files[decode_alg], DEFAULT_COLORS[decode_alg]

        axes[i].set_title(decode_alg)
        ax = sns.histplot(data, palette=palette, ax=axes[i], **hist_kwargs)
            
        dec_algs, dec_avgs = compute_avgs(data, property1, "decoding")
        for alg, avg in zip(dec_algs, dec_avgs):
            ax.axvline(avg, label=alg, color=palette[alg], ls="--")
            
    plt.tight_layout()

In [ ]:
plot_histograms(property1="sequence_log_prob", binwidth=10) # must normalize size of the properties
plt.show()
plot_histograms(property1="sequence_log_prob", binwidth=10) # must normalize size of the properties
plt.xlim(-400, 0)
plt.show()

In [ ]:
plot_histograms(property1="num_chars", binwidth=50)
plt.show()
plot_histograms(property1="num_chars", binwidth=50)
plt.xlim(0, 650)
plt.show()

In [ ]:
plot_histograms(property1="num_sentences", binwidth=1)
plt.show()
plot_histograms(property1="num_sentences", binwidth=1)
plt.xlim(0, 12)
plt.show()

In [ ]:
plot_histograms(property1="num_punct", binwidth=5)
plt.show()
plot_histograms(property1="num_punct", binwidth=1)
plt.xlim(0, 40)
plt.show()

In [ ]:
plot_histograms(property1="num_words", binwidth=5)
plt.show()
plot_histograms(property1="num_words", binwidth=2)
plt.xlim(0, 125)
plt.show()

In [ ]:
plot_histograms(property1="diversity", binwidth=0.025)
plt.show()
plot_histograms(property1="diversity", binwidth=0.025)
plt.xlim(0.15, 0.85)
plt.show()

In [ ]:
plot_histograms(property1="toxicity", binwidth=0.02)
plt.show()
plot_histograms(property1="toxicity", binwidth=0.02)
plt.ylim(0, 0.001)
plt.show()

## Quantiles

In [ ]:
import sys
sys.path.append("..")
from quantiles import quantile_intervals
from collections import defaultdict

import itertools as it

QUANTILES = [0.1, 0.5, 0.75, 0.9, 0.95, 0.99]
QUANTILES_CI_CONFIDENCE = 0.95

# Sanity check
r = quantile_intervals(len(files[0]), QUANTILES, desired_confidence=QUANTILES_CI_CONFIDENCE)
r

In [ ]:
files[0]["toxicity"].sort_values().values[r["desired_ranks"].astype(int)]

In [ ]:
def compute_quantiles(
        decodings,
        files,
        properties=["toxicity", "diversity", "num_sentences", "sequence_log_prob"],
        quantiles=QUANTILES,
        conf=QUANTILES_CI_CONFIDENCE,
    ):
    def add_base_info(r, algorithm, quants):
        r["decoding"].extend([algorithm] * len(quants))
        r["quantiles"].extend(quants.tolist())
    
    results = defaultdict(list)
    results_lo_rank = defaultdict(list)
    results_up_rank = defaultdict(list)
    
    print(len(decodings), len(files))
    # Iterate each decoding algorithm
    for dalgo, file in zip(decodings, files):
        print("Computing quantiles for", dalgo, "on", len(file), "examples")
        q_intervals = quantile_intervals(len(file), quantiles, conf)
        
        # Base information
        add_base_info(results, dalgo, q_intervals["desired_quantiles"])
        add_base_info(results_lo_rank, dalgo, q_intervals["desired_quantiles"])
        add_base_info(results_up_rank, dalgo, q_intervals["desired_quantiles"])

        # For each property, compute results, lower interval and upper_interval
        for prop1 in properties:
            val = file[prop1].sort_values().values
            
            # Measure rank
            q_ranks = q_intervals["desired_ranks"].astype(int)
            results[prop1].extend(val[q_ranks])
            
            # Intervals
            q_lranks = q_intervals["lower_interval_ranks"].astype(int)
            results_lo_rank[prop1].extend(val[q_lranks])
            
            q_uranks = q_intervals["upper_interval_ranks"].astype(int)
            results_up_rank[prop1].extend(val[q_uranks])
        
    return pd.DataFrame(results), pd.DataFrame(results_lo_rank), pd.DataFrame(results_up_rank)
    # return results, results_lo_rank, results_up_rank

print("Computing quantiles!")
quantiles_results, quantiles_lresults, quantiles_uresults = compute_quantiles(decodings, files)
quantiles_results.head()

In [ ]:
def plot_quantiles(quantiles_results, colname: str="toxicity"):
    fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=150)
    sns.scatterplot(quantiles_results, y="decoding", x=colname, hue="quantiles", ax=ax)
    plt.legend(title="Quantile", bbox_to_anchor=(1.25, 1), borderaxespad=0)
    plt.title(MODEL)
    
plot_quantiles(quantiles_results, "toxicity")
plt.xscale("log")
plt.show()

### Quantiles with error bars

In [ ]:
QUANTILE_PALETTE = sns.color_palette("colorblind", n_colors=len(QUANTILES))
QUANTILE_COLORS = {q: color for color, q in zip(QUANTILE_PALETTE, QUANTILES)}


def plot_quantiles_w_error_bars(quantiles, low_quantiles, up_quantiles, colname="toxicity", qs=QUANTILES, color_by_quantiles=QUANTILE_COLORS, figsize=(10, 10)):
    """Plot the quantiles and the error bars as given by low_quantiles and up_quantiles.
    
    Parameters
    ----------
    quantiles: pandas.DataFrame
        The data containing the values for different decoding algorithms
        and quantiles
    
    low_quantiles: pandas.DataFrame
        The absolute lower bound of the quantiles for different decoding
        algorithms,
        
    up_quantiles: pandas.DataFrame
        The absolute upper bound of the quantiles for different decoding
        algorithms. We convert it to relative automatically.
        
    qs: list[float]
        The list with the desired quantiles to map. Make sure you
        specify quantile values that are present in the data you
        provide and for which the color has been defined as well.
    """
    def get_quantile_subset(data, q):
        return data[data["quantiles"] == q].copy()

    fig, ax = plt.subplots(1, 1, figsize=figsize)
    errs = []
    # Iterate for each different value of quantile
    for q in qs:
        # Pick a common color
        qcolor = color_by_quantiles[q]
        # Select a slice of the data that is specific to that quantile
        qresults = get_quantile_subset(quantiles, q)
        # Select the same slice from the lower quantile
        qlresults = get_quantile_subset(low_quantiles, q)
        # Select the slice from the upper quantile
        quresults = get_quantile_subset(up_quantiles, q)
        # The size of these slices should be the same
        assert len(qresults) == len(qlresults) == len(quresults)
        
        # Generate some example data
        x = qresults[colname].values
        y = qresults["decoding"].values
        y_err = np.vstack((
            qresults[colname].values - qlresults[colname].values,
            quresults[colname].values - qresults[colname].values,
        ))
        # Create the scatterplot with error bars
        plt.errorbar(x, y, xerr=y_err, fmt='o', capsize=2, color=qcolor, label=q, alpha=0.7)
        errs.append(y_err)

    # Show the plot
    plt.title(f"[{MODEL}] {colname}")
    plt.legend(title="Quantile", bbox_to_anchor=(1.15, 1), borderaxespad=0)
    
    
plot_quantiles_w_error_bars(
    quantiles=quantiles_results,
    low_quantiles=quantiles_lresults,
    up_quantiles=quantiles_uresults,
    colname="toxicity",
    qs=QUANTILES, 
)
plt.xscale("log")

In [ ]:
quantiles_results

In [ ]:
plot_quantiles_w_error_bars(
    quantiles=quantiles_results,
    low_quantiles=quantiles_lresults,
    up_quantiles=quantiles_uresults,
    colname="sequence_log_prob",
    qs=QUANTILES, 
)

In [ ]:
decodings = quantiles_results["decoding"]
quantiles = quantiles_results["quantiles"]
results = quantiles_results["toxicity"]
lresults = quantiles_lresults["toxicity"]
uresults = quantiles_uresults["toxicity"]

for d in zip(decodings, quantiles, lresults, results, uresults):
    print(d, d[-2] - d[-3], d[-1] - d[-2])


In [ ]:
y_err

In [ ]:
# Generate some example data
x = np.array([1, 2, 3, 4, 5])
y = np.array([1, 3, 2, 4, 5])
y_err = np.array([[0.0, 0.2, 0.35, 0.56, 0.78], [0.5, 0.7, 0.4, 0.6, 0.8]])

# Create the scatterplot with error bars
plt.errorbar(x, y, yerr=y_err, fmt='o')

# Add labels and title
plt.xlabel('X-axis label')
plt.ylabel('Y-axis label')
plt.title('Scatterplot with Custom Error Bars')

# Show the plot
plt.show()

In [ ]:
colname: str="toxicity"
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=150)
sns.scatterplot(quantiles_results, y="decoding", x=colname, hue="quantiles", ax=ax)
plt.legend(title="Quantile", bbox_to_anchor=(1.25, 1), borderaxespad=0)

plt.title(MODEL)
plt.xscale("log")
plt.show()

In [ ]:
colname = "toxicity"

# sns.pointplot(data=quantiles_results, x=colname, y="decoding", join=False)

In [ ]:
quantiles_results

In [ ]:
plot_quantiles(quantiles_results, "diversity")

## Joint distributions

In [ ]:
def plot_joints(decodings, files, x="toxicity", y="diversity", bins=(50, 50), rang=((0,1), (0,1))):
    for dalgo, file in zip(decodings, files):
        fig, ax = plt.subplots(1,1, figsize=(7, 5))
        plt.title(f"[{MODEL}: {dalgo}]")
        ax.hist2d(x=file[x], y=file[y], bins=(100, 20), cmap=plt.cm.Reds, density=True, range=rang)
        ax.set_xlabel(x)
        ax.set_ylabel(y)
        plt.show()

In [ ]:
plot_joints(decodings, files, "toxicity", "diversity", bins=(100, 50))

### Barplot

In [ ]:
def barplot(data_all, by: str, colname):
    data_all.groupby(by).count()[colname].plot(kind="bar")
    # plt.gca().xaxis.grid(False)


In [ ]:
barplot(all_files_concat[all_files_concat["toxicity"] > 0.5], "decoding", "toxicity")

## Greedy-like sequences

In [ ]:
from rich.console import Console
from rich.text import Text

# Console for printing with nice colors :)
console = Console(width=80)

ALL_COLORS = {}
for colors_dict in DEFAULT_COLORS.values():
    for k, v in colors_dict.items():
        ALL_COLORS[k] = v
        
def pprint(text, model_score, decoding, counts=0):
    msg  = f"\n\n({decoding}) {model_score:.2%} Toxicity (generated {counts} times /15000)\nText: [cyan]'{text}'[/cyan]"
    console.print(msg)
    
    
def pprint_df(df: pd.DataFrame, colname: str, n=None):
    df_ndup = df.drop_duplicates("sequence").reset_index(drop=True).copy()

    for i, row in df_ndup.iterrows():
        n_times = (df["sequence"] == row["sequence"]).sum()
        pprint(row["sequence"], row[colname], row["decoding"], n_times)
        
        if n is not None and i > n:
            print(i, n)
            break

In [ ]:
temp_greedy = files[decodings.index("temperature_0.1")].sort_values("toxicity", ascending=False)
print("Temperature=0.1: |S| =", temp_greedy['sequence'].nunique())

top_p_greedy  = files[decodings.index("top_p_0.1")].sort_values("toxicity", ascending=False)
print("Top-p=0.1: |S| =", top_p_greedy['sequence'].nunique())

top_k_greedy  = files[decodings.index("top_k_2")].sort_values("toxicity", ascending=False)
print("Top-k=2: |S| =", top_k_greedy['sequence'].nunique())

In [ ]:
pprint_df(top_p_greedy, "toxicity", 1)

In [ ]:
pprint_df(temp_greedy, "toxicity", 50)

In [ ]:
pprint_df(top_k_greedy, "toxicity", 50)

# Worst case analysis

### Toxicity Analysis

In [ ]:
bottom_files = [f.sort_values("toxicity", ascending=False).head(150) for f in files]
bottom_files_concat = pd.concat(bottom_files, axis=0)
bottom_quantiles = compute_quantiles(decodings, bottom_files)

In [ ]:
plot_quantiles(bottom_quantiles, "toxicity")
plot_quantiles(bottom_quantiles, "diversity")

In [ ]:
plot_joints(decodings, bottom_files, "toxicity", "diversity", bins=(50, 50), rang=((0,1), (0,1)))